In [1]:
#https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-7-phrase-modeling-doc2vec-592a8a996867
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import re
import os
print(os.getcwd())

#Read Sheet 1
columns = ['','Date', 'Time', 'Tweet', 'Class', '', '' ]
df1 = pd.read_excel("trainingObamaRomneytweets.xlsx", sheet_name="Romney",header=None, skiprows=2, encoding='utf-8', names=columns)
pd.options.display.max_colwidth=10000
dfTrain = pd.DataFrame()
dfTrain['tweet'] = df1['Tweet']
dfTrain['class'] = df1['Class']#, 'Class'], axis=1)



E:\Varsha\Sem 2\Data Mining\RomneyVsObama


In [2]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
negationsA_dic = {"isn't":"not", "aren't":"not", "wasn't":"not", "weren't":"not",
                "haven't":"not","hasn't":"not","hadn't":"not","won't":"not",
                "wouldn't":"not", "don't":"not", "doesn't":"not","didn't":"not",
                "can't":"not","couldn't":"not","shouldn't":"not","mightn't":"not",
                "mustn't":"not", "cant":"not", "cmon":"come on", "couldnt":"not", 
                "youre": "you are", "atax":" a tax", "wen":"when", "statthement":"statement",
                "doe":"do", "ththem":"them", "em":"them", "cn":"can", "abt":"about",
                "omg":"oh my god", "lookin":"looking", "talkin":"talking", "il":"i will",
                "caidates":"candidates", "romneyjust": "romney just", "arou":"around",
                "gon na":"going to", "gonna":"going to"}

sadEmoji = [":\(", ":-\(", ":|", ":-|", ":/", ":-/", ":\\", ":-\\", ":-\]"]#, ":\\", ":-\\", ":-\]"]
happyEmoji = [':\)', ':-\)', '<3', ':*', ':-*', ':D', ':-D']
neg_pattern = re.compile(r'\b(' + '|'.join(negationsA_dic.keys()) + r')\b')


def tweet_cleaner_updated(text):
    sadProcessed = text
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
#     try:
#         bom_removed = souped.decode("utf-8").replace(u"\ufffd", "?")
#         print("From Try Block")
#     except:
#         bom_removed = souped
    text = re.sub('@[^\s]+','',text)
    #text = " ".join(text)
    #print(text)
    neg_handled = neg_pattern.sub(lambda x: negationsA_dic[x.group()], text)
    return neg_handled #letters_only #(" ".join(words)).strip()

In [3]:
#FROM PREVIOUS CODE
def clean1(df):
    df["tweet"]=df["tweet"].astype(str)
    df["tweet"] = df["tweet"].str.strip() #Strips trailing and leading spaces
    df["tweet"]=df["tweet"].str.replace("$|<e>|</e>|<a>|</a>|\(|\)|<|\-|lâ€¦|â€™|<|>|\'|\?|\_|\”|\“", "")
    pat2 = r'https?://[^ ]+'
    pat3 = r'http://[^ ]+'
    www_pat = r'www.[^ ]+'
    users_pat = r'@[^ ]+'
    for pattern in sadEmoji:
             df['tweet'].replace(pattern, "sad", inplace=True)
    for pattern in happyEmoji:
             df['tweet'].replace(pattern, "happy", inplace=True)
    df['tweet'] = df['tweet'].str.replace("&|amp", " and ")
    df['tweet'] = df['tweet'].str.replace(pat2, "")
    df['tweet'] = df['tweet'].str.replace(pat3, "")
    df['tweet'] = df['tweet'].str.replace(www_pat, "")
    df['tweet'] = df['tweet'].str.replace(users_pat, "")
    df["tweet"]=df["tweet"].str.replace(r"[^a-zA-Z ]+", "").str.strip() #Only alphabets are retained, Removing @ improves accuracy by 1%-53%
    df["tweet"]=df["tweet"].str.lower()
    df['tweet'] = df['tweet'].str.replace("rd|eh|va|nd|hea|pfffft|rt", "")#Converts to lowercase
    #df["tweet"]=df["tweet"].str.replace('[^\w\s]','') #Removes punctuations
    print("Completed First Level Cleaning.")
    return df

In [4]:
def clean2(df):
    for index, row in df.iterrows():
        row["tweet"] = tweet_cleaner_updated(row["tweet"])
     
    df["tweet"].replace('', np.nan, inplace=True)

        
    df[pd.notnull(df["tweet"])]
    df = df[df.tweet.notnull()]

    df = df[df["tweet"]!=" "]
    df = df.dropna()

    df["tweet"]=df["tweet"].astype(str)
  
        
    
    print("Completed Second Level Cleaning.")
    return df

In [5]:
def clean2class(df):
    df["class"] = df["class"].astype(str)
    df = df[(df["class"] != "2")]
    df["class"].replace('', np.nan, inplace=True)
    df[pd.notnull(df["class"])]
    df = df[df["class"] != "nan"]
    df = df[(df["class"] != "NaN")]
    df = df[df["class"] != "!!!!"]
    df = df[df["class"] != ""]
    df = df[(df["class"] != "irrevelant")]
    df = df[(df["class"] != "irrelevant")]
    df = df[df["class"] != " "]
    df = df[df["class"]!="IR"]
    df.dropna(subset=["class", "tweet"], inplace=True)
    df = df.dropna()
    df["class"] = df["class"].astype(str)
    print(len(df.index))
    return df

In [6]:
# freq = pd.Series(" ".join(df['tweet']).split()).value_counts()[:50]  #Common words removal
# freq = list(freq.index)
# df['tweet'] = df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

# freq = pd.Series(" ".join(df['tweet']).split()).value_counts()[-10:]   #Infrequent words Removal
# freq = list(freq.index)
# df['tweet'] = df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# df['tweet']=df['tweet'].astype(str)
# df['class'] = df['class'].astype(str)
# df.to_csv("E:/Varsha/Sem 2/Data Mining/RomneyVsObama/trainingObamaRomneytweetsProcessed.csv")

In [7]:
#Text Spelling Corrections


In [8]:
from sklearn.model_selection import train_test_split, GroupKFold, KFold, cross_val_score
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from nltk.tokenize import word_tokenize
#from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, precision_score, make_scorer, accuracy_score
import nltk
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer 
from imblearn.over_sampling import SMOTE
from textblob import Word
#from imblearn.pipeline import make_pipeline, Pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from imblearn.combine import SMOTEENN
from collections import Counter
from imblearn.over_sampling import SMOTE

def finalClean(df):
    lemmatizer = WordNetLemmatizer()


    df["tweetList"]=df["tweet"].apply(word_tokenize) #Tokenize to words and join to form sentences using NLTK
    df["tweet"]=df["tweetList"].apply(lambda x: " ".join(map(str, x)))

    def lemmatize_text(text):
        return [lemmatizer.lemmatize(w) for w in nltk.word_tokenize(text)]

    df['tweetList'] = df.tweet.apply(lemmatize_text)
    df["tweet"]=df["tweetList"].apply(lambda x: " ".join(map(str, x)))

    df['tweet'] = df['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

    nltk.download('stopwords')
    from nltk.corpus import stopwords #Reduces accuracy by 2%
    stop = stopwords.words('english')
    df['tweet']=df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

    from nltk.stem import LancasterStemmer
    lancaster=LancasterStemmer() #Lancaster Stemming
    df["tweetList"].apply(lambda x: [lancaster.stem(y) for y in x])
    df['tweet']=df["tweetList"].apply(lambda x: " ".join(map(str, x)))

    #FINAL CLEANING

    df["tweet"].replace('', np.nan, inplace=True)

    df = df.dropna()
        
    df[pd.notnull(df["tweet"])]
    df['tweet'] = df['tweet'].str.replace("@ ", "@")
    df['tweet'] = df['tweet'].str.replace("# ", "#")
    print(df.index)
    #df.to_csv("trainingObamaRomneytweetsProcessed.csv")

    # label = np.array(df['class'])
    # data = np.array(df['tweet'])
    
        
    # count_vect = CountVectorizer(analyzer='word', binary=True, ngram_range=(1,2))
    # count_vect = count_vect.fit(df['tweet'])
    # tf_transformer = TfidfVectorizer(count_vect)
    # tf_transformer.fit(df['tweet'])
    print("Completed Final Level Cleaning.")
#    df = df.sample(frac=1, axis=1).sample(frac=1).reset_index(drop=True) #Shuffle
    return df

In [9]:
def finalcleanclass(df):
    df["class"].replace('', np.nan, inplace=True)
    df[pd.notnull(df["class"])]
#     encoder = preprocessing.LabelEncoder()
#     df['class']=encoder.fit_transform(df['class'])
    df['class']=np.array(df['class'])
    return df

In [10]:
from sklearn.externals import joblib
from imblearn.over_sampling import SMOTE
dfTrain = clean1(dfTrain)
dfTrain = clean2(dfTrain)
dfTrain = clean2class(dfTrain)
dfTrain = finalClean(dfTrain)
dfTrain = finalcleanclass(dfTrain)
trainTweet, testTweet, trainClass, testClass = train_test_split(dfTrain['tweet'], dfTrain['class'], test_size=0.2)
#df.to_csv("trainingObamaRomneytweetsProcessed.csv")
#SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42

sm = SMOTE(ratio = 'auto',k_neighbors = 5, n_jobs = -1)

#sm = SMOTE()
#smoteModel = SMOTEENN(smote=sm)
model = Pipeline([('bow', CountVectorizer(strip_accents='ascii',
                            stop_words='english',
                            lowercase=True)),  
#         ('smote_enn', sm),
    ('tfidf', TfidfTransformer()),
                          ('clf', svm.SVC(kernel="linear"))]
)
                                               
parameters = {'bow__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__C': [0.1,0.5,1,1.2, 1.5]
             }
kf = StratifiedKFold(n_splits=10, shuffle=True)
grid = GridSearchCV(model, param_grid=parameters, n_jobs=-1, verbose=100, cv=10)
grid.fit(trainTweet.astype(str), trainClass.astype(int))
predClass = grid.predict(testTweet)
joblib.dump(grid, "twitter_sentiment.pkl")
print(len(predClass))
print('accuracy score: ',accuracy_score(testClass.astype(int), predClass.round().astype(int)))
print("Best params: ", grid.best_params_)
print("Paras: ", grid.get_params().keys())
print("Classification matrix: ", classification_report(testClass.astype(int), predClass.round().astype(int)))

Completed First Level Cleaning.
Completed Second Level Cleaning.
5648


C:\Users\varsh\Anaconda3\lib\site-packages\pandas\core\generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\varsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Int64Index([   0,    2,    3,    4,    5,    6,    7,   10,   12,   13,
            ...
            7187, 7188, 7189, 7190, 7191, 7192, 7195, 7196, 7198, 7199],
           dtype='int64', length=5648)
Completed Final Level Cleaning.
Fitting 10 folds for each of 20 candidates, totalling 200 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4065,), dtype=int32).
Pickling array (shape=(453,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64)

[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   13.5s
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4066,), dtype=int32).
Pickling array (shape=(452,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   13.5s
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4066,), dtype=int32).
Pickling array (shape=(452,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   17.6s
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pic

[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:   26.7s
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4066,), dtype=int32).
Pickling array (shape=(452,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   26.7s
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4066,), dtype=int32).
Pickling array (shape=(452,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   26.9s
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pic

...

[Parallel(n_jobs=-1)]: Done 117 tasks      | elapsed:  1.3min
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4066,), dtype=int32).
Pickling array (shape=(452,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 118 tasks      | elapsed:  1.3min
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4066,), dtype=int32).
Pickling array (shape=(452,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 119 tasks      | elapsed:  1.3min
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pic

[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  1.6min
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4066,), dtype=int32).
Pickling array (shape=(452,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.6min
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
[Parallel(n_jobs=-1)]: Done 139 tasks      | elapsed:  1.6min
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4066,), dtype=int32).
Pickling array (shape=(452,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pic

[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:  2.0min
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4066,), dtype=int32).
Pickling array (shape=(452,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.0min
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4066,), dtype=int32).
Pickling array (shape=(452,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 159 tasks      | elapsed:  2.0min
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pic

[Parallel(n_jobs=-1)]: Done 177 tasks      | elapsed:  2.5min
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4066,), dtype=int32).
Pickling array (shape=(452,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:  2.5min
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=int32).
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4066,), dtype=int32).
Pickling array (shape=(452,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 179 tasks      | elapsed:  2.5min
Pickling array (shape=(4518,), dtype=int64).
Pickling array (shape=(4518,), dtype=object).
Pic

C:\Users\varsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:

grid = joblib.load("twitter_sentiment.pkl" )

ObamaRowIdList = []
RomneyRowIdList = []

testTweet_Obama =pd.read_csv("Obama_Test_dataset_NO_Label_Encoded.csv",header=None, skiprows=2, encoding='utf-8', names=['Tweet_ID', 'Tweet_text'])
dfTest_Obama = pd.DataFrame()
dfTest_Obama['rowId'] = testTweet_Obama['Tweet_ID']
dfTest_Obama['tweet']= np.array(testTweet_Obama['Tweet_text'])
dfTest_Obama = clean1(dfTest_Obama)
dfTest_Obama = clean2(dfTest_Obama)
dfTest_Obama = finalClean(dfTest_Obama)

predClass_Obama = grid.predict(dfTest_Obama['tweet'])
#print(len(predClass_Obama))

testTweet_Romney =pd.read_csv("Romney_Test_dataset_NO_Label_Encoded.csv",header=None, skiprows=2, encoding='utf-8', names=['Tweet_ID', 'Tweet_text'])
dfTest_Romney = pd.DataFrame()
dfTest_Romney['rowId'] = testTweet_Romney['Tweet_ID']
dfTest_Romney['tweet']= np.array(testTweet_Romney['Tweet_text'])
dfTest_Romney = clean1(dfTest_Romney)
dfTest_Romney = clean2(dfTest_Romney)
dfTest_Romney = finalClean(dfTest_Romney)
#print(len(dfTest_Romney.index))
predClass_Romney = grid.predict(dfTest_Romney['tweet'])

print("Completed Prediction.")

dfTest_Obama.to_csv("E:/Varsha/Sem 2/Data Mining/RomneyVsObama/trainingObamaRomneytweetsProcessedObama.csv")
dfTest_Romney.to_csv("E:/Varsha/Sem 2/Data Mining/RomneyVsObama/trainingObamaRomneytweetsProcessedRomney.csv")
ObamaRowIdList = list(dfTest_Obama['rowId'])
RomneyRowIdList = list(dfTest_Romney['rowId'])
#print(len(predClass_Obama), len(ObamaRowIdList))
ObamaFile = open("Varsha_Jayaraman_Obama.txt","a")
RomneyFile = open("Varsha_Jayaraman_Romney.txt","a")
# Predictions from best model above

#print(len(y_preds))
for i in range(0,len(predClass_Obama)):
    str1 = str(ObamaRowIdList[i]) + ';;' + str(predClass_Obama[i])
    #print(str1)
    ObamaFile.write(str1 + "\n")
ObamaFile.close()

for i in range(0,len(predClass_Romney)):
    str1 = str(RomneyRowIdList[i]) + ';;' + str(predClass_Romney[i])
    #print(str1)
    RomneyFile.write(str1 + "\n")
RomneyFile.close()
# outfile = open(r'result2.txt','w+')
# for i in range(len(dfTest.index)):
#     print(ids[i], ';;', yPredicted[i], sep='', file=outfile)
# outfile.close()

Completed First Level Cleaning.
Completed Second Level Cleaning.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\varsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949],
           dtype='int64', length=1950)
Completed Final Level Cleaning.
Completed First Level Cleaning.
Completed Second Level Cleaning.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\varsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1889, 1890, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898],
           dtype='int64', length=1899)
Completed Final Level Cleaning.
Completed Prediction.


In [31]:
if("class" in df.columns):
    print("a")

a


In [61]:
predClass_Obama

array([0, 0, 0])